In [14]:
import pandas as pd
import random
import math
import numpy as np
from scipy.interpolate import interp1d

In [15]:
pu239s=pd.read_csv('Pu-239 scattering.csv')
pu239f=pd.read_csv('Pu-239 fission.csv')
pu239a=pd.read_csv('Pu-239 absorption.csv')

atom_density=6.02*10**23*19.84/239

x=10**-6*np.array(pu239f['Incident energy'])
y=10**-24*atom_density*np.array(pu239f['Cross section'])
pf=interp1d(x,y)

x=10**-6*np.array(pu239a['Incident energy'])
y=10**-24*atom_density*np.array(pu239a['Cross section'])
pa=interp1d(x,y)

x=10**-6*np.array(pu239s['Incident energy'])
y=10**-24*atom_density*np.array(pu239s['Cross section'])
ps=interp1d(x,y)

In [16]:
def absorbtion_cross_section(geometry,neutron_energy):
	neutron_energy=neutron_energy
	if geometry=='p':
		return pa(neutron_energy)
	else:
		return 0
	


def scattering_cross_section(geometry,neutron_energy):
	neutron_energy=neutron_energy
	if geometry=='p':
		return ps(neutron_energy)
	else:
		return 0

def fission_cross_section(geometry,neutron_energy):
	neutron_energy=neutron_energy
	if geometry=='p':
		return pf(neutron_energy)
	else:
		return 0
		
def energy_updater(energy,molar_mass):
	A=molar_mass
	E=energy
	alpha=1-((A-1)/(A+1))**2
	delta_E=np.random.uniform(0,alpha*E)
	
	return E-delta_E

def watt_spectrum(energy):
    
    prob=0.4865*np.sinh(np.sqrt(2*energy))*np.exp(-energy) 
    
    return prob


def prompt_neutron_energy():
    a=0.0001
    b=14
    max=0.3574479547952136

    
    logic=True

    while logic==True:

        bal=np.random.rand()*(b-a)+a
        rand_num=np.random.rand()

        if rand_num*max<=watt_spectrum(energy=bal):
            break
    return float(bal)			


In [17]:
skip=2
number_of_neutrons=1000
no_gen_sim=10				      
position_matrix=np.zeros((number_of_neutrons,3))
energy=[]
new_gen_neutron=np.zeros(no_gen_sim)
weight0=np.ones(number_of_neutrons)
weight1=np.ones(number_of_neutrons)

for i in range (number_of_neutrons):
	position_matrix[i][0]=np.random.uniform(0,6.3849)                    #radius record 
	position_matrix[i][1]=np.random.uniform(-np.pi,np.pi)                #theta record 
	position_matrix[i][2]=np.random.uniform(-np.pi/2,np.pi/2)	  		 #phi record 
	energy.append(np.random.uniform(2,4))


k_eff=np.zeros(no_gen_sim)


In [18]:
for m in range (no_gen_sim):


	number_of_neutrons=1000			      
	position_matrix=np.zeros((number_of_neutrons,3))
	energy=[]


	for i in range (number_of_neutrons):
		position_matrix[i][0]=np.random.uniform(0,6.3849)                    #radius record 
		position_matrix[i][1]=np.random.uniform(-np.pi,np.pi)                #theta record 
		position_matrix[i][2]=np.random.uniform(-np.pi/2,np.pi/2)	  		 #phi record 
		energy.append(prompt_neutron_energy())

	k_eff__=np.zeros(10)

	for j in range(10):
		
		r=[]
		theta=[]
		phi=[]
		energy_new=[]

		new_gen_neutron=np.zeros(10)
			
		for i in range (number_of_neutrons):
			
			#location copy
			x=position_matrix[i][0]*math.sin(position_matrix[i][2])*math.cos(position_matrix[i][1])
			y=position_matrix[i][0]*math.sin(position_matrix[i][2])*math.sin(position_matrix[i][1])
			z=position_matrix[i][0]*math.cos(position_matrix[i][2])
			Energy = energy[i]
			alive=True
			
			while alive:
			
			# core 
				if math.sqrt(x**2+y**2+z**2)<6.3849:
				
				#that means the neutron is inside the core 
					
					sigma_s=scattering_cross_section("p",neutron_energy=Energy)
					sigma_a=absorbtion_cross_section("p",neutron_energy=Energy)
					sigma_f=fission_cross_section("p",neutron_energy=Energy)
					
					total_sigma=sigma_s+sigma_f+sigma_a
					
					reaction_type=np.random.choice(["fission","scattering","absorbtion"],p=[sigma_f/total_sigma,sigma_s/total_sigma,sigma_a/total_sigma])
					
					#need to figure out how to make a bias here ------done
					
					if reaction_type =="scattering":
						
						random_parameter=np.random.rand()
						theta_move=random.uniform(-np.pi,np.pi)
						phi_move=random.uniform(0,np.pi)
						
						l= -math.log(1-random_parameter)/total_sigma
						
						x = x+l*math.sin(phi_move)*math.cos(theta_move)
						y = y+l*math.sin(phi_move)*math.sin(theta_move)
						z = z+l*math.cos(phi_move)
						
						Energy =energy_updater(Energy,239)
							
					elif reaction_type=="fission":
						
						neu=2.5
						meu=np.random.rand()
						if meu<(neu-int(meu)):
							new_neutron=int(neu)
						else: 
							new_neutron=int(neu)+1

						new_gen_neutron[j]=new_gen_neutron[j]+new_neutron
						
						#location has to be saved for new initiation 
						#there will be a loop to create the same location in a matrix
						#for the same number of neutron 
						
						for k in range(new_neutron):
						
					
							r.append(math.sqrt(x**2+y**2+z**2))
							theta.append(math.atan(y/x))
							phi.append(math.atan(math.sqrt(x**2+y**2)/z))
							
							energy_new.append(prompt_neutron_energy())
						
						alive=False 
					else:
						alive=False 
				else: 
					alive=False

		position_matrix=np.zeros((len(r),3))
		
		for k in range(len(r)):

			position_matrix[k][0]=r[k]
			position_matrix[k][1]=theta[k]
			position_matrix[k][2]=phi[k]

			

			
			
		k_eff__[j]=(new_gen_neutron[j]/number_of_neutrons)
		energy=np.zeros(len(energy_new))
		energy=energy_new 

	
	a=np.average(k_eff__[5:])
	print(m,'--',a)
	k_eff[m]=a

	
print(np.mean(k_eff),'-/+',np.std(k_eff))

0 -- 1.1751999999999998
1 -- 1.1816
2 -- 1.2036
3 -- 1.1887999999999999
4 -- 1.2004
5 -- 1.2036
6 -- 1.1691999999999998
7 -- 1.1812
8 -- 1.206
9 -- 1.16
1.1869599999999998 -/+ 0.015306155624453883


In [19]:
import openmc

# Create plutonium metal material
pu = openmc.Material()
pu.add_nuclide('Pu239',1)
pu.set_density(density=19.84,units='g/cm3')

mats = openmc.Materials([pu])
mats.export_to_xml()

# Create a single cell filled with the Pu metal
sphere = openmc.Sphere(r=6.3849, boundary_type='vacuum')
cell = openmc.Cell(fill=pu, region=-sphere)
geom = openmc.Geometry([cell])
geom.export_to_xml()

# Finally, define some run settings
settings = openmc.Settings()
settings.batches = 200
settings.inactive = 10
settings.particles = 1000
settings.export_to_xml()

# Run the simulation
openmc.run()
n = settings.batches
with openmc.StatePoint(f'statepoint.{n}.h5') as sp:
    keff = sp.keff
    print(f'Final k-effective = {keff}')

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

In [27]:

print('Throin','%.6f'%np.mean(k_eff),'-/+','%.6f'%np.std(k_eff))
print('openmc',keff)

Throin 1.186960 -/+ 0.015306
openmc 1.2362+/-0.0017
